In [1]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-S", trust_remote_code=True)
model = AutoModel.from_pretrained("zhihan1996/DNABERT-S", trust_remote_code=True)

/nethome/rfeng44/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-S/1cdf84d992ace6f3e75c7356774b4da088c8dc7c/bert_layers.py:125: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


In [3]:
import pandas as pd
filtered_dataset = pd.read_feather("../data/filtered_dataset.ftr")

tokenized = tokenizer(filtered_dataset['sequence'].tolist())

In [7]:
tokenized['input_ids'][0]

[1,
 3887,
 289,
 87,
 81,
 754,
 61,
 40,
 230,
 73,
 81,
 219,
 613,
 770,
 970,
 262,
 231,
 22,
 141,
 59,
 89,
 931,
 95,
 124,
 89,
 29,
 119,
 78,
 615,
 2635,
 838,
 627,
 51,
 50,
 31,
 620,
 89,
 157,
 21,
 71,
 80,
 2184,
 2]

In [2]:
import pandas as pd

master_dataset = pd.read_feather("../data/master_dataset.ftr")

In [3]:
sequences = master_dataset["sequence"].values

In [4]:
import sys
def custom_progress_bar(sequence):
    for i, item in enumerate(sequence):
        sys.stdout.write(r"\rProcessing {i+1}/{len(sequence)}")
        yield item
    sys.stdout.write("\n")

In [5]:
len(sequences[0]), len(sequences)

(200, 3591898)

In [6]:
!pwd

/localscratch/rfeng44/vecint/notebooks


In [7]:

batch_size = 512
output_file = open("../data/sequence_embeddings.txt", "w")
from tqdm.notebook import tqdm

device = 'cuda:0'

model.to(device)

for sequence_batch in tqdm(range(0, len(sequences), batch_size)):
    batch = sequences[sequence_batch:sequence_batch+batch_size].tolist()
    inputs = tokenizer(batch, return_tensors="pt", padding=True)

    inputs.to(device)
    outputs = model(**inputs)
    embeddings = outputs[0]
    print(embeddings.shape)
    for i, sequence in enumerate(batch):
        output_file.write(f"{sequence}\t{embeddings[i].detach().cpu().numpy()}\n")

  0%|          | 0/7016 [00:00<?, ?it/s]

torch.Size([512, 50, 768])


OutOfMemoryError: CUDA out of memory. Tried to allocate 74.00 MiB. GPU 0 has a total capacity of 23.68 GiB of which 57.88 MiB is free. Including non-PyTorch memory, this process has 23.62 GiB memory in use. Of the allocated memory 22.43 GiB is allocated by PyTorch, and 972.37 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)